In [1]:
import openai
import os
def get_openai_key():
    # Check if the file exists
    if not os.path.isfile('openai_key.txt'):
        # Create the file if it does not exist and write a default value or leave it blank
        with open('openai_key.txt', 'w') as file:
            file.write('')  # You could prompt the user for a key or leave it blank
    # Read the key from the file
    with open('openai_key.txt', 'r') as file:
        return file.read().strip()


# Usage
openai.api_key = get_openai_key()

In [2]:
import re

def build_html_from_compact_representation(compact_rep, component_name=None):
    item_count = 0

    def create_element(tag, class_name=None, placeholder=None, data_type=None, content=""):
        nonlocal item_count
        attributes = []
        if class_name:
            attributes.append(f'class="{class_name}"')
        if placeholder:
            attributes.append(f'placeholder="{placeholder}"')
        if data_type:
            item_id = f"galleryItem_{component_name}_{item_count}" if component_name else f"newItem_{item_count}"
            attributes.extend([f'data-type="{data_type}"', 'draggable="true"', 'type="button"', f'id="{item_id}"'])
            item_count += 1
        if tag == "input":
            attributes.append('style="border: none;box-shadow: none;text-align: center;"')
        return f"<{tag} {' '.join(attributes)}>{content}</{tag}>"

    def process_compact_rep(compact_string):
        html = []
        pattern = r'(\w+)(?:\[([^\]]+)\]|\s*{)'
        matches = list(re.finditer(pattern, compact_string))

        i = 0
        while i < len(matches):
            match = matches[i]
            type_, content = match.groups()
            
            if content and '}' not in content:  # Simple element
                if type_ in ["title", "label"]:
                    div_class = "btn-primary btn-tutor-title" if type_ == "title" else "btn-success btn-label row"
                    dt = "page-items" if type_ == "title" else "page-row"
                    p_content = create_element("p", "page-item", None, None, content)
                    p_content += create_element("p", "removeFromDom", None, None, "Tutor Title" if type_ == "title" else "Label")
                    html.append(create_element("div", f"btn {div_class} rounded-button", None, dt, p_content))
                elif type_ == "input":
                    input_elem = create_element("input", "form-control", content, None)
                    html.append(create_element("div", "btn btn-light btn-input-box-t rounded-button", None, "page-row", input_elem))
            else:  # Complex element (row or column)
                div_type = "page-row" if type_ == "row" else "page-items"
                div_class = "btn-warning btn-row btn-row-item grid" if type_ == "row" else "btn-info btn-column"
                p_remove = create_element("p", "removeFromDom", None, None, type_.capitalize())
                ul_class = "list one page-item-ul grid grid-flow-col gap-1" if type_ == "row" else "list one d-flex flex-column gap-2 page-item-rl-row"
                
                # Find the content for this element
                start = match.end()
                end = find_matching_brace(compact_string, start)
                if end == -1:
                    end = len(compact_string)
                inner_content = compact_string[start:end]
                
                ul_content = process_compact_rep(inner_content)
                ul = f'<ul class="{ul_class}" data-type="page-row">{"".join(ul_content)}</ul>'
                html.append(create_element("div", f"btn {div_class} drop-box rounded-button", None, div_type, p_remove + ul))
                
                # Move to the next top-level element
                i = next((j for j in range(i+1, len(matches)) if matches[j].start() > end), len(matches))
                continue

            i += 1

        return html

    def find_matching_brace(string, start):
        stack = 0
        for i in range(start, len(string)):
            if string[i] == '{':
                stack += 1
            elif string[i] == '}':
                stack -= 1
                if stack == 0:
                    return i
        return -1

    html_parts = process_compact_rep(compact_rep)
    page_div = create_element("div", None, None, "page-items", "".join(html_parts))
    final_container_div = create_element("div", "container mx-auto flex flex-col py-1 justify-center items-center", None, None, "")
    return page_div + final_container_div




In [3]:
import re

from bs4 import BeautifulSoup
from bs4 import BeautifulSoup

def process_element(element):
    compact_rep = ''
    class_list = element.get('class', [])
    
    if 'btn-tutor-title' in class_list:
        title = element.find('p').get_text(strip=True)
        compact_rep += f"title[{title}]"
    elif 'btn-row' in class_list:
        child_elements = element.select('ul > li')
        if child_elements:
            compact_rep += 'row{'
            compact_rep += ', '.join(process_element(child) for child in child_elements)
            compact_rep += '}'
        else:
            compact_rep += 'row{}'
    elif 'btn-column' in class_list:
        child_elements = element.select('ul > li')
        if child_elements:
            compact_rep += 'column{'
            compact_rep += ', '.join(process_element(child) for child in child_elements)
            compact_rep += '}'
        else:
            compact_rep += 'column{}'
    elif 'btn-label' in class_list:
        label = element.find('p').get_text(strip=True)
        compact_rep += f"label[{label}]"
    elif 'btn-input-box-t' in class_list:
        input_placeholder = element.find('input').get('placeholder', 'Text Box')
        compact_rep += f"input[{input_placeholder}]"
    
    return compact_rep

def compact_html_representation(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    elements = soup.find_all(True, class_=['btn-tutor-title', 'btn-row', 'btn-column', 'btn-label', 'btn-input-box-t'])
    
    # Process top-level elements
    representation = []
    for element in elements:
        if not any(parent.get('class', []) & {'btn-row', 'btn-column'} for parent in element.parents):
            representation.append(process_element(element))
    
    return ', '.join(representation)


In [4]:
import re

def convert_format(input_string):
    # Step 1: Remove spaces and split the string into components
    components = re.split(r'\]\s*', input_string.strip(']'))

    # Step 2: Initialize an empty list to hold nested structure
    stack = []

    # Helper function to close nested structures
    def close_nested():
        nested = []
        while stack and stack[-1] != '{':
            nested.append(stack.pop())
        if stack and stack[-1] == '{':
            stack.pop()  # pop the '{'
        return nested[::-1]

    # Step 3: Process each component
    for comp in components:
        if 'row column' in comp:
            # Special case: row followed by column
            stack.append('row{')
            stack.append('column{')
        elif 'row' in comp:
            # Start a new row
            stack.append(comp + '{')
        elif 'column' in comp:
            # Start a new column
            stack.append(comp + '{')
        elif 'input' in comp or 'label' in comp:
            # Add input/label component
            stack.append(comp)
        else:
            # General case
            stack.append(comp)
            if '[' in comp:
                stack.append('{')

    # Step 4: Close all open nested structures
    nested = close_nested()
    while nested:
        if len(nested) > 1:
            component = ', '.join(nested)
        else:
            component = nested[0]
        if stack and stack[-1].endswith('{'):
            stack[-1] += component + '}'
        nested = close_nested()

    # Step 5: Construct the final output string
    final_output = 'Compact Representation Layout String:\n' + ', '.join(stack)
    return final_output



In [5]:
def generate_layout(description= 'a tutor for solving one rational equation'):
  detailed_description = description
  # Introduction to the task
  prompt_system = "Create a dynamic and interactive tutor interface layout specifically crafted for problem-solving exercises, which commences with the precise definition of variables that are essential for the problem at hand. Each interface is to facilitate a clear, step-by-step resolution pathway, commencing with an explicit problem statement and progressing through a structured series of steps. Each step is to include text inputs that are directly linked to data sources, ensuring their correctness can be validated through an HTN (Hierarchical Task Network) process. This setup should lead to the revealing of the final solution, all while adhering to pedagogical best practices that logically sequence problem-solving elements and enforce the understanding of the educational content."

  prompt_introduction = """Generate a compact representation layout string for a tutor interface based on a specific format. This format includes elements such as titles, rows, columns, labels, and inputs."""
  # Explanation of the format
  prompt_format = "The format uses: title[Title], row{{...}}, column{{...}} with each element stacked vertically over the other within the column, label[Label], and input[Placeholder]. Elements within rows and columns are enclosed in curly braces {{}}, and attributes are in square brackets []."
  # Design instructions for the layout
  prompt_design_instructions = """Design principles require that each input element be separate, for example, in an equation, there should be one input element per digit. A single equation must be on a single row. Elements within a row are arranged horizontally. Rows cannot be directly nested within other rows, nor can columns be nested within other columns. Ensure each input element is separate, such as digits in an equation, and a single equation appears on a single line. There are no interactive buttons like 'Click to solve' within the layout."""
  # Instruction to transform the description into the layout string
  prompt_task = """Transform the detailed description into the compact representation layout string according to the instructions."""
  prompt_examples = "Example 0: A fraction is always of the form column{input[Numerator], label[____], input[Denominator]} and is never row{input[Numerator], label[/], input[Denominator]}"
  prompt_examples += "Example 1: Equation row with two fraction members should be represented as a row with multiple columns inside for the members. Each column contains stacked numerator, operand (division symbol), and denominator. Include labels for operators in the main row. For a fraction equation like 1/2 + 3/4, the layout should be row{column{input[Numerator], label[____], input[Denominator]}, label[+], column{input[Numerator], label[____], input[Denominator]}, label[=], input[Result]}. This format ensures that each part of the fraction equation is clearly defined and separated for input. \
  "
  # Examples of the prompt format
  prompt_examples += ", Example 2: A two equations solver should be represented as title[2 Equation Solver], column{label[First Equation], row{input[First Equation Coefficient 1], label[x], label[+], input[First Equation Coefficient 2], label[=], input[First Equation Result]}, label[Second Equation], row{input[Second Equation Coefficient 1], label[x], label[+], input[Second Equation Coefficient 2], label[y], label[=], input[Second Equation Result]}}."
  prompt_examples += ", Example 3: For a radicals tutor interface, the representation would be title[Radicals Tutor], column{label[Solve the following radicals multiplication problem below], row{label[What is √2 * √2], input[Result]}, input{label[Simplify the expression]}}"
  prompt_examples += ", Example 4: For a Squared Tutor interface, the representation would be title[Squared Tutor], column{label[Enter the number you wish to square], row{input[Number], label[=], input[Result]}}. This configuration places the label and input for the number and the result all in a single horizontal row, maintaining a clear and concise layout."
  prompt_examples += ", Example 5: For a Tutor for missionaries and cannibals, the representation would be title[Missionaries and Cannibals Tutor], column{label[Instructions], row{label[Enter the number of missionaries], input[Missionaries]}, row{label[Enter the number of cannibals], input[Cannibals]}, label[Solution], row{input[First Move], label[->], input[Second Move]}, row{input[Third Move], label[->], input[Fourth Move]}}. As in This case, make sure to scaffold all the resolution steps."
  prompt_examples += ", Example 6: For a tutor for calculating proper drug dosage levels for a nurse to administer to a patient, the representation would be title[Drug Dosage Tutor], column{label[Calculation Instructions], row{label[Enter the weight of the patient (in kg)], input[Patient Weight]}, row{label[Enter the recommended dosage per kg (in mg)], input[Dosage per kg]}, label[Calculated Dosage], row{input[Dosage Total], label[mg]}}"
  prompt_examples += ", Example 7: For A tutor for optimizing a business workflow, the representation would be title[Business Workflow Optimization Tutor], column{label[Instructions], row{label[Enter the number of employees], input[Number of Employees]}, row{label[Enter the number of hours worked by each employee per week], input[Hours Worked]}, row{label[Enter the average number of tasks completed by an employee per hour], input[Average Tasks Completed]}, label[Optimization Solution], row{input[Current Workflow Efficiency], label[%]}, row{label[Enter the changes you wish to make], input[Suggested Changes]}, label[Calculated Optimized Efficiency], row{input[Optimized Workflow Efficiency], label[%]}}"
  prompt_examples += ", Example 8: For an English article selection tutor, the representation would be title[English Article Selection Tutor], column{label[Instructions], label[Select the most appropriate article ('a', 'an', 'the' or 'no article') for each blank in the sentences below], row{label[Sentence 1], input[Sentence 1 Article Choice]}, row{label[Sentence 2], input[Sentence 2 Article Choice]}, row{label[Sentence 3], input[Sentence 3 Article Choice]}}"
  prompt_examples += ", Example 9: For a tutor for conducting a cash flow analysis of a business, the representation would be title[Cash Flow Analysis Tutor], column{label[Instructions], label[Enter the relevant business data in the fields below to begin your cash flow analysis conversion], row{label[Enter the total revenue of the business for the chosen period], input[Total Revenue]}, row{label[Enter the total cost of goods sold (COGS) during the same period], input[Cost of Goods Sold]}, label[Gross Profit], row{input[Gross Profit], label[]}, row{label[Enter the total operating expenses during the same period], input[Operating Expenses]}, label[Net Profit], row{input[Net Profit], label[]}, row{label[Enter the total cash flows from investing activities (e.g. purchase of assets, investments)], input[Investing Cash Flows]}, row{label[Enter the total cash flows from financing activities (e.g. loans, dividends, repayable)], input[Financing Cash Flows]}, label[Net Cash Flow], row{input[Net Cash Flow], label[]}}"
  prompt_examples += ", Example 9: For a tutor for a three members rational equation, the representation would be title[Rational Equations], column{label[Solve the rational equation], row{column{input[Numerator 1], label[___], input[Denominator 1]}, label[+], column{input[Numerator 2], label[___], input[Denominator 2]}, label[=], column{input[Numerator 3], label[___], input[Denominator 3]}}}"


  # Combining all the parts to form the complete prompt
  prompt = f"{prompt_system}\n{prompt_introduction}\n\n{prompt_format}\n\n{prompt_design_instructions}\n\n{prompt_examples}\n\n{prompt_task}"
  instruction = f"\nDetailed Description:\n{detailed_description}\n"

  response = openai.chat.completions.create(
    model="gpt-4", # Adjusted to use GPT-4
  messages=[{"role": "system", "content": prompt}, {"role": "user", "content": instruction }]
  )
  generated_text = response.choices[0].message.content.strip()
  return generated_text
  # Return the high-level steps generated by OpenAI


In [6]:
from bs4 import BeautifulSoup

def process_element(element, processed=None):
    if processed is None:
        processed = set()
    
    if element in processed:
        return ''
    
    processed.add(element)
    
    class_list = element.get('class', [])
    
    if 'btn-tutor-title' in class_list:
        title = element.find('p', class_='page-item').get_text(strip=True)
        return f"title[{title}]"
    elif 'btn-row' in class_list or 'btn-column' in class_list:
        container_type = 'row' if 'btn-row' in class_list else 'column'
        return process_container(element, container_type, processed)
    elif 'btn-label' in class_list:
        label = element.find('p', class_='page-item').get_text(strip=True)
        return f"label[{label}]"
    elif 'btn-input-box-t' in class_list:
        input_placeholder = element.find('input').get('placeholder', 'Text Box')
        return f"input[{input_placeholder}]"
    return ''

def process_container(element, container_type, processed):
    children = element.find('ul', recursive=False).find_all('div', recursive=False)
    if children:
        child_elements = [process_element(child, processed) for child in children]
        child_elements = [elem for elem in child_elements if elem]  # Remove empty strings
        return f"{container_type}{{{', '.join(child_elements)}}}"
    return f"{container_type}{{}}"

def compact_html_representation(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    top_container = soup.find('div', attrs={'data-type': 'page-items'})
    
    if top_container:
        elements = top_container.find_all('div', recursive=False)
        processed = set()
        representation = ', '.join(filter(None, (process_element(elem, processed) for elem in elements)))
        return representation
    else:
        return "No top-level container found"



In [15]:
def create_element(tag, class_name=None, placeholder=None, data_type=None, content=""):
    global item_count
    attributes = ""
    if class_name:
        attributes += f' class="{class_name}"'
    if placeholder:
        attributes += f' placeholder="{placeholder}"'
    if data_type:
        attributes += f' data-type="{data_type}" draggable="true" type="button" id="newItem_{item_count}"'
        item_count += 1
    if tag == "input":
        attributes += ' style="border: none;box-shadow: none;text-align: center;"'
    return f'<{tag}{attributes}>{content}</{tag}>'

In [80]:
import execjs

# Define your JavaScript function
js_code = """
  let item_count = 0;

  function buildHTMLFromCompactRepresentation(compactRep, componentName) {

    // Function to create an HTML string for an element with optional class, placeholder, and additional attributes
    function createElement(tag, className, placeholder, dataType, content = "") {
      let attributes = "";
      if (className) attributes += ` class="${className}"`;
      if (placeholder) attributes += ` placeholder="${placeholder}"`;
      if (dataType) {
        if (componentName) {
          attributes += ` data-type="${dataType}" draggable="true" type="button" id="galleryItem_${componentName}_${item_count++}"`;
        } else {
          attributes += ` data-type="${dataType}" draggable="true" type="button" id="newItem_${item_count++}"`;
        }
      }
      if (tag === "input") {
        attributes += ` style="border: none;box-shadow: none;text-align: center;"`;
      }
      return `<${tag}${attributes}>${content}</${tag}>`;
    }

    // Function to recursively process and generate HTML string from the compact representation
    function appendElementsFromCompactRep(compactString, isChild = true) {
      const regex = /(\w+)\[([^\]]+)\]|\w+\s*{/g;
      let match,
        html = "";

      while ((match = regex.exec(compactString)) !== null) {
        const [matchedString, type, content] = match;
        if (type && content) {
          switch (type) {
            case "title":
            case "label":
              const divClass =
                type === "title"
                  ? "btn-primary btn-tutor-title"
                  : "btn-success btn-label row";
              const dt =
                type === "title"
                  ? "page-items"
                  : "page-row";
              const p = createElement("p", "page-item", null, null, content) + createElement("p", "removeFromDom", null, null, "Tutor Title")
              const q = createElement("p", "page-item align-self-center", null, null, content) + createElement("p", "removeFromDom", null, null, "Label")
              const f = type === "title" ? p : q;
              html += createElement(
                "div",
                `btn ${divClass} rounded-button`,
                null,
                dt,
                f
              );
              break;
            case "input":
              const input = createElement("input", "form-control", content, null);
              input.style = "border: none;box-shadow: none;text-align: center;";
              html += createElement(
                "div",
                "btn btn-light btn-input-box-t rounded-button",
                null,
                "page-row",
                input
              );
              break;
          }
        } else if (matchedString.trim().endsWith("{")) {
          const elementType = matchedString.trim().replace("{", "").trim();
          const divType = elementType === "row" ? "page-row" : "page-items";
          const divClass =
            elementType === "row"
              ? "btn-warning btn-row btn-row-item grid"
              : "btn-info btn-column";
          const pRemove = createElement(
            "p",
            "removeFromDom",
            null,
            null,
            elementType.charAt(0).toUpperCase() + elementType.slice(1)
          );
          const ulClass =
            elementType === "row"
              ? "list one page-item-ul grid grid-flow-col gap-1"
              : "list one d-flex flex-column gap-2 page-item-rl-row";
          const ulId =
            elementType === "row" ? "page-item-ul" : "page-item-rl-row";
          const closeIndex = findClosingIndex(compactString, regex.lastIndex);
          const innerContent = compactString.substring(
            regex.lastIndex,
            closeIndex
          );
          const ulContent = appendElementsFromCompactRep(innerContent, true);
          const ul = `<ul class="${ulClass}" data-type="page-row">${ulContent}</ul>`;

          html += createElement(
            "div",
            `btn ${divClass} drop-box rounded-button`,
            null,
            divType,
            pRemove + ul
          );
          regex.lastIndex = closeIndex + 1;
        }
      }
      if (isChild) {
        return html;
      } else {
        const pageDiv = createElement("div", null, null, "page-items", html);
        const finalContainerDiv = createElement(
          "div",
          "container mx-auto flex flex-col py-1 justify-center items-center",
          null,
          null,
          ""
        );
        return html + finalContainerDiv;
      }
    }

    function findClosingIndex(str, openIndex) {
      let stack = 1;
      for (let i = openIndex; i < str.length; i++) {
        if (str[i] === "{") stack++;
        else if (str[i] === "}") stack--;

        if (stack === 0) return i;
      }
      return -1; // Handle malformed strings gracefully
    }
    html = appendElementsFromCompactRep(compactRep)
    return html;
  }
"""

# Compile the JavaScript code
context = execjs.compile(js_code)



In [9]:
layout = generate_layout()


In [10]:
layout

'Compact Representation Layout String:\ntitle[Rational Equation Solver], column{label[Solve the given rational equation], row{column{input[Numerator 1], label[___], input[Denominator 1]}, label[=], input[Result]}}'

In [72]:
item_count = 0

def create_element(tag, class_name=None, placeholder=None, data_type=None, content=""):
    global item_count
    attributes = ""
    if class_name:
        attributes += f' class="{class_name}"'
    if placeholder:
        attributes += f' placeholder="{placeholder}"'
    if data_type:
        attributes += f' data-type="{data_type}" draggable="true" type="button" id="newItem_{item_count}"'
        item_count += 1
    if tag == "input":
        attributes += ' style="border: none;box-shadow: none;text-align: center;"'
    return f'<{tag}{attributes}>{content}</{tag}>'

def find_closing_index(compact_string, start_index):
    count = 1
    for i in range(start_index, len(compact_string)):
        if compact_string[i] == "{":
            count += 1
        elif compact_string[i] == "}":
            count -= 1
            if count == 0:
                return i
    return -1


In [93]:
def append_elements_from_compact_rep(compact_string, is_child=False):
    import re

    regex = re.compile(r"(\w+)\[([^\]]+)\]|\w+\s*{")
    match = None
    html = ""
    last_index = 0

    while True:
        match = regex.search(compact_string, last_index)
        if not match:
            break

        matched_string, element_type, content = match.group(0), match.group(1), match.group(2)
        last_index = match.end()

        if element_type and content:
            if element_type in ["title", "label"]:
                div_class = "btn-primary btn-tutor-title" if element_type == "title" else "btn-success btn-label row"
                dt = "page-items" if element_type == "title" else "page-row"
                p_content = content if element_type == "title" else content
                p_attrs = {"class": "page-item"} if element_type == "title" else {"class": "page-item align-self-center"}
                p = create_element("p", p_content, p_attrs)
                p_remove_attrs = {"class": "removeFromDom"}
                p_remove_content = "Tutor Title" if element_type == "title" else "Label"
                p_remove = create_element("p", p_remove_content, p_remove_attrs)
                div_content = p + p_remove
                div_attrs = {"class": f"btn {div_class} rounded-button", "data-type": dt}
                html += create_element("div", div_content, div_attrs)
            elif element_type == "input":
                input_attrs = {
                    "class": "form-control",
                    "style": "border: none;box-shadow: none;text-align: center;"
                }
                input_element = create_element("input", None, input_attrs)
                div_attrs = {"class": "btn btn-light btn-input-box-t rounded-button", "data-type": "page-row"}
                html += create_element("div", input_element, div_attrs)
        elif matched_string.strip().endswith("{"):
            element_type = matched_string.strip().replace("{", "").strip()
            div_type = "page-row" if element_type == "row" else "page-items"
            div_class = "btn-warning btn-row btn-row-item grid" if element_type == "row" else "btn-info btn-column"
            div_attrs = {"class": f"btn {div_class} drop-box rounded-button", "data-type": div_type}
            p_remove_attrs = {"class": "removeFromDom"}
            p_remove_content = element_type.capitalize()
            p_remove = create_element("p", p_remove_content, p_remove_attrs)
            ul_class = "list one page-item-ul grid grid-flow-col gap-1" if element_type == "row" else "list one d-flex flex-column gap-2 page-item-rl-row"
            ul_attrs = {"class": ul_class, "data-type": "page-row"}
            closing_index = find_closing_index(compact_string, last_index)
            inner_content = compact_string[last_index:closing_index]
            ul_content = append_elements_from_compact_rep(inner_content, True)
            ul = create_element("ul", ul_content, ul_attrs)
            div_content = p_remove + ul
            html += create_element("div", div_content, div_attrs)
            last_index = closing_index + 1

    if is_child:
        return html
    else:
        page_div_attrs = {"data-type": "page-items"}
        page_div_content = html
        page_div = create_element("div", page_div_content, page_div_attrs)
        final_container_div_attrs = {"class": "container mx-auto flex flex-col py-1 justify-center items-center"}
        final_container_div_content = ""
        final_container_div = create_element("div", final_container_div_content, final_container_div_attrs)
        return page_div + final_container_div




In [94]:
layout = 'title[Rational Equation Solver], column{label[Solve the given rational equation], row{column{input[Numerator 1], label[___], input[Denominator 1]}, label[=], input[Result]}}'
html = append_elements_from_compact_rep(layout.split('\n')[-1])

In [95]:
html

'<div class="<div class="<p class="Rational Equation Solver" placeholder="{\'class\': \'page-item\'}"></p><p class="Tutor Title" placeholder="{\'class\': \'removeFromDom\'}"></p>" placeholder="{\'class\': \'btn btn-primary btn-tutor-title rounded-button\', \'data-type\': \'page-items\'}"></div><div class="<p class="Column" placeholder="{\'class\': \'removeFromDom\'}"></p><ul class="<div class="<p class="Solve the given rational equation" placeholder="{\'class\': \'page-item align-self-center\'}"></p><p class="Label" placeholder="{\'class\': \'removeFromDom\'}"></p>" placeholder="{\'class\': \'btn btn-success btn-label row rounded-button\', \'data-type\': \'page-row\'}"></div><div class="<p class="Row" placeholder="{\'class\': \'removeFromDom\'}"></p><ul class="<div class="<p class="Column" placeholder="{\'class\': \'removeFromDom\'}"></p><ul class="<div class="<input placeholder="{\'class\': \'form-control\', \'style\': \'border: none;box-shadow: none;text-align: center;\'}" style="bor

In [81]:
result = context.call("buildHTMLFromCompactRepresentation", layout.split('\n')[-1], )


In [69]:
html == result

False

In [83]:
result

'<div class="btn btn-primary btn-tutor-title rounded-button" data-type="page-items" draggable="true" type="button" id="newItem_0"><p class="page-item">Rational Equation Solver</p><p class="removeFromDom">Tutor Title</p></div><div class="btn btn-info btn-column drop-box rounded-button" data-type="page-items" draggable="true" type="button" id="newItem_9"><p class="removeFromDom">Column</p><ul class="list one d-flex flex-column gap-2 page-item-rl-row" data-type="page-row"><div class="btn btn-success btn-label row rounded-button" data-type="page-row" draggable="true" type="button" id="newItem_1"><p class="page-item align-self-center">Solve the given rational equation</p><p class="removeFromDom">Label</p></div><div class="btn btn-warning btn-row btn-row-item grid drop-box rounded-button" data-type="page-row" draggable="true" type="button" id="newItem_8"><p class="removeFromDom">Row</p><ul class="list one page-item-ul grid grid-flow-col gap-1" data-type="page-row"><div class="btn btn-info bt